In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = 15, 8
import warnings
warnings.simplefilter('ignore')

# visualización de las primeras n imágenes en formato mnist (imágenes vectorizadas de 28x28)
def visu_mnist(X, n=10):    
    visu = [X[i].reshape((28,28)) for i in range(n)]
    visu = np.vstack([np.hstack(visu[:n//2]), np.hstack(visu[n//2:])])
    visu = (visu - visu.min()) / (visu.max() - visu.min())
    plt.imshow(visu, cmap='gray')

In [ ]:
from sklearn.datasets import fetch_mldata
# Descargamos el dataset en caso de que no haya sido descargado previamente -> aprox. 56MB
# Como alternativa el mismo puede ser descargado manualmente desde 
# https://drive.google.com/file/d/12E2XQSaVi-pQVCIxEPbgiZtTFo81Fo0K/view?usp=sharing
mnist = fetch_mldata('MNIST original', data_home='/home/ezequiel/datasets/')

# Dividimos el dataset en un conjunto de entrenamiento (80% de los datos) y test (20% restantes)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, train_size=0.8)
print('{} train samples with {} dimensions'.format(X_train.shape[0], X_train.shape[1]))
print('{} test samples with {} dimensions'.format(X_test.shape[0], X_test.shape[1]))
    
visu_mnist(X_train)

In [ ]:
# Ejemplo de un punto en el conjunto de entrenamiento
print(X_train.shape)
print(y_train.shape)

print(X_train[0])
plt.imshow(X_train[0].reshape((28,28)), cmap='gray')
plt.show

In [ ]:
print(y_train[0])

### 2.2 Dimentional Reduction (Reducción de dimensionalidad)
Un gran problema que surge cuando tratamos de realizar algún tipo de **aprendizaje automático** es la **maldición de la dimensionalidad**, la cual se refiere a los diversos fenómenos que surgen al analizar y organizar datos en espacios de múltiples dimensiones (cientos y miles de dimensiones) que no suceden en el espacio físico descrito generalmente con solo tres dimensiones.

Sitio donde hay una buena explicación sobre el tema ([link](http://www.albertolumbreras.net/posts/maldicion-dimensionalidad.html)) 

El problema con las grandes dimensiones, es que cuanto mas dimensiones hay, todos los puntos quedan concentrados en un espacio muy pequeño. En otras palabras, todos los puntos estarán cerca. Esto se puede visualizar en la siguiente imagen:

![betancourt_cubes](img/11-betancourt_cubes.png)

Por esta razón, se han investigado técnicas, para poder proyectar datos en un espacio de dimensionalidad $D$ en otro espacio de dimensionalidad menor $M$ tal que $M\leq D$. Una de las técnicas mas utilizadas realizar esto es el **Análisis de Componentes Principales (PCA)**.

#### Principal Component Analysis (PCA)
En PCA lo que se busca es encontrar una matriz de proyección $U$ tal que describir un set de datos en términos de nuevas variables, llamadas **componentes**, no correlacionadas. Estos componentes se ordenan por la cantidad de varianza que describen. Permitiendo seleccionar una cantidad $p$ de componentes principales para describir el dataset que explique el conjunto de datos.

![pca](img/11-basic_pca.png)

##### Algoritmo: 

Dato el conjunto de datos de entrenamiento $\{x_1, \dots, x_n\}\mid x_i \in R^D$
* Computar la media $\bar{x}$ para cada dimensión y la matriz de covarianza $S$
$$\bar{x}=\frac{1}{N}\sum_{n=1}^N x_n\qquad S=\frac{1}{N-1}\sum_{n=1}^N(x_n-\bar{x})^T(x_n-\bar{x})$$
* Calcular los auto-valores $(\lambda_i\mid i\in D)$ y auto-vectores $(u_i\mid i\in D)$ de la matriz de covarianza $S$
* Ordenar los auto-vectores $u_i$ de manera decreciente con respecto a sus correspondientes auto-valores $\lambda_i$.
* Crear la matriz $U=(u_1,\dots,u_D)$

Para realizar la proyección al nuevo espacio ortogonal:
$$x_n^D=(U^T(x_n-\bar{x})^T)^T$$

$x_n^D$ es la matriz de ejemplos transformados ortogonalmente. A partir de la misma, podemos descartar dimensiones para llevar nuestro dataset a un espacion de $M$ dimensiones. 

Para recontruir una aproximación de los datos originales
$$\tilde{x}_n\approx x_n^MU^T+\bar{x}$$

El error cuadratico de reconstrucción es
$$\sum_{n=1}^N(x_n-\tilde{x}_n)^2=(N-1)\sum_{j=M+1}^D\lambda_j$$

donde los $\lambda_j$ son los auto-valores descartados en la proyección.

##### ¿Como elegir M?
$$\frac{\sum_{k=1}^M\lambda_k}{\sum_{k=1}^D\lambda_k}>thresh$$ 
tal que $(0\leq thresh \leq 1)$.

El $min(M)$ que satisfaga la ecuación previa explicara el $thresh$ porciento de los datos.

In [ ]:
# Proyecto los datos escalados con PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=128)
pca.fit(X_train_scaled)

In [ ]:
plt.bar(range(pca.explained_variance_ratio_.shape[0]), pca.explained_variance_ratio_)
plt.show()
#print(pca.explained_variance_ratio_)  

In [ ]:
# visualizar las primeras componentes principales (columnas de U)
visu_mnist(pca.components_, 20)

In [ ]:
# Finalmente, transformamos con PCA los datos de entrenamiento
X_train_pca = pca.transform(X_train_scaled)

## Ejercicios 
1) Implementar PCA

In [ ]:
def pca_train(X):       
    # 1) Implementar PCA mediante descomposición de la matriz de 
    # covarianza muestral. Utilizar la función np.linalg.eigh
    # para una solución numéricamente más estable. Los valores 
    # a retornar son una matriz con las componentes principales 
    # como columnas (U) y el vector medio (mean). 
    # NOTA: las columnas de U tienen que retornar en orden de 
    # "importancia" decreciente 
    #

    
    return U, mean

def pca_project(X, U, mean, keep_dim=-1):
    n_samples, n_dim = X.shape
    assert n_dim == len(mean)
    if keep_dim < 0:
        keep_dim = n_dim

    # 2) Implementar la proyección de puntos en X (filas)
    # empleando los (U, mean) estimados a partir del conjunto 
    # de entrenamiento. El parámetro keep_dim es el número de
    # componentes a considerar (proyección de n_dim a keep_dim 
    # dimensiones). Si este valor es menor a 0, la dimensionalidad 
    # del espacio de salida es la misma que el de entrada.
    #X_proj = ...
    
    return X_proj

def pca_restore(X_proj, U, mean):
    n_samples, keep_dim = X_proj.shape
    return X_proj.dot(U[:, :keep_dim].T) + mean.reshape(1, -1)


3) Volver a entrenar un predictor para el dataset **MNIST original** utilizando la implementación propia de PCA y K-Means.